<a href="https://colab.research.google.com/github/subvarshahere/codsoft/blob/main/credit_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import shutil
import pandas as pd
import zipfile
from google.colab import files
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:

from google.colab import files
files.upload()
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.move("kaggle(2).json", "/root/.kaggle/")
os.chmod("/root/.kaggle/kaggle.json", 600)
!pip install kaggle
!kaggle datasets download -d mlg-ulb/creditcardfraud
with zipfile.ZipFile("creditcardfraud.zip", "r") as zip_ref:
    zip_ref.extractall("creditcardfraud")  # Extract to a folder
df = pd.read_csv("creditcardfraud/creditcard.csv")
df.head()




Saving kaggle(2).json to kaggle(2).json
Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
creditcardfraud.zip: Skipping, found more recently modified local copy (use --force to force download)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
#  Preprocess the data

X = df.drop(columns=['Class', 'Time'])
y = df['Class']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

# Logistic Regression
#lr = LogisticRegression()
#lr.fit(X_train, y_train)
#y_pred_lr = lr.predict(X_test)

# Decision Tree
#dt = DecisionTreeClassifier()
#dt.fit(X_train, y_train)
#y_pred_dt = dt.predict(X_test)

# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)


In [5]:
# models
def evaluate_model(y_test, y_pred):
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

#print("\nLogistic Regression Evaluation:")
##evaluate_model(y_test, y_pred_lr)

#print("\nDecision Tree Evaluation:")
#evaluate_model(y_test, y_pred_dt)

print("\nRandom Forest Evaluation:")
evaluate_model(y_test, y_pred_rf)

smote = SMOTE(sampling_strategy='minority', random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)
rf_resampled = RandomForestClassifier()
rf_resampled.fit(X_res, y_res)
y_pred_rf_resampled = rf_resampled.predict(X_test)

print("\nRandom Forest with SMOTE Evaluation:")
evaluate_model(y_test, y_pred_rf_resampled)



Random Forest Evaluation:
Accuracy: 0.9995786664794073
ROC AUC Score: 0.9030172599428067
Confusion Matrix:
[[56859     5]
 [   19    79]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.94      0.81      0.87        98

    accuracy                           1.00     56962
   macro avg       0.97      0.90      0.93     56962
weighted avg       1.00      1.00      1.00     56962


Random Forest with SMOTE Evaluation:
Accuracy: 0.9994908886626171
ROC AUC Score: 0.9131597912096746
Confusion Matrix:
[[56852    12]
 [   17    81]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.87      0.83      0.85        98

    accuracy                           1.00     56962
   macro avg       0.94      0.91      0.92     56962
weighted avg       1.00      1.00      1.00     56962

